# J-Quants APIを用いた株式分析チュートリアル

本ノートブックは、2部構成になっています。
前半で日本取引所グループが提供している [株式分析チュートリアル](https://japanexchangegroup.github.io/J-Quants-Tutorial/) の実行に必要となる株価・財務情報データを [J-Quants API](https://jpx-jquants.com/#jquants-api)を使用して取得し、データを調整し、保存するところまでを行います。

後半では、取得したデータを用いて[J-Quants株式分析チュートリアル第2章のハンズオンノートブック](https://github.com/JapanExchangeGroup/J-Quants-Tutorial/blob/main/handson/Chapter02/20210121-chapter02-tutorial.ipynb) の再現を行います。

Google Colab上で動作確認を行っています。

---

**このノートブックはGoogle Driveを使用します。**

- Google Drive の以下のフォルダーにデータを書き込みます。
    - `MyDrive/drive_ws/marketdata`

In [ ]:
# Googleドライブをマウントするディレクトリ
GOOGLE_DRIVE_MOUNT_DIR_PATH = "/content/drive"

# Googleドライブをマウント
from google.colab import drive
drive.mount(GOOGLE_DRIVE_MOUNT_DIR_PATH)

# データ作成

必要なライブラリのinstall/importと設定を行います。

本ノートブックでは、J-Quants API のPythonクライアントライブラリである [jquants-api-client-python](https://github.com/J-Quants/jquants-api-client-python) を使用します。

In [ ]:
# jquants-api-client をインストールします
!pip install jquants-api-client

import getpass
import os
from datetime import datetime
from typing import List

import numpy as np
import pandas as pd
from requests import HTTPError

import jquantsapi

# --- コンフィグ ---

# データを保存するGoogleドライブ上のディレクトリ
STORAGE_DIR_PATH = f"{GOOGLE_DRIVE_MOUNT_DIR_PATH}/MyDrive/drive_ws/marketdata"

# 各種CSVデータを保存するファイルパス
# 元データ
raw_stock_list_csvfile_path = f"{STORAGE_DIR_PATH}/raw_stock_list.csv.gz"
raw_stock_fins_csvfile_path = f"{STORAGE_DIR_PATH}/raw_stock_fin.csv.gz"
raw_stock_price_csvfile_path = f"{STORAGE_DIR_PATH}/raw_stock_price.csv.gz"
# 処理済みデータ
stock_list_csvfile_path = f"{STORAGE_DIR_PATH}/stock_list.csv.gz"
stock_fins_csvfile_path = f"{STORAGE_DIR_PATH}/stock_fin.csv.gz"
stock_price_csvfile_path = f"{STORAGE_DIR_PATH}/stock_price.csv.gz"
stock_labels_csvfile_path = f"{STORAGE_DIR_PATH}/stock_labels.csv.gz"

# J-Quants API から取得するデータの期間
start_dt: datetime = datetime(2018, 1, 1)
end_dt: datetime = datetime(2023, 3, 31)

# 目的変数(ラベル)の作成時に、何営業日後の価格レンジを予測するか。
# たとえばlookaheads = [5, 10, 20] であった場合、5営業日後、10営業日後, 20営業日後
# の高値・安値を取得し、stock_labels変数には下記のカラムが付与されたdataframeが保存されます。
# "label_date_5", "label_high_5", "label_low_5", 
# "label_date_10", "label_high_10", "label_low_10",
# "label_date_20", "label_high_20", "label_low_20",
# lookaheads = [5, 10, 20]
lookaheads = [20] # 数が多いと目的変数作成に時間がかかるので一旦20だけにする

# データ保存先のフォルダーを作成しておきます
os.makedirs(STORAGE_DIR_PATH, exist_ok=True)

In [ ]:
refresh_token = "eyJjdHkiOiJKV1QiLCJlbmMiOiJBMjU2R0NNIiwiYWxnIjoiUlNBLU9BRVAifQ.LusUmC14JF3LGVD2R84u1_W7W3rqorfa0QMoxNiAMrAfNbBiJFbhovoL73L96_Og-vwq50r1HNuvK7TIMxyb3F5VC1cconM-WYCSFDfI7cGKOEfulzj9Q0wbhPheEsUtZyjNfBOncrv-eosPFAT4l9_E0YHRjEVQC-QpUWMsbouu2LGCMyzNnRE_cTTN7iFYr9lgXLBM5-6GGUtCvz-s-rfNMVkrYJOu_uvLyVSbx0CIdd2-sMi7PBdSyltEqQMmpmjIofAmoXu_YEcXGl32NqPO6LlXY_z7goAeFD61ojPeIR_pLpjCSCESn8LeQlz4UfpiDtH5OxiRKssffswXDQ.UHu8mSoJl7ZFsp9k.ljpCshZE-KIgng_YMBqk8omrgf_EyXvtMte6IFK4E4pjw1eHthO3G2JKKpJWd64YjTyKOFkilU0TeInM2iLNXdiUwbNMNAkrGA1etBzfmdoJdfJ9FPaBoZCZkx4ylTcsdh5KSR0s-YLfasQiIXal9NiGbbIhY3GFZ8F56wOmXd3tLKkDEda-psgytM96qUz6ZGxOBNRrIqBufKGJci_waAwIl2xZ8eMj4xm9HuWyiDFXM5ezOPxRje803INuU25CFyxs8yQTB3MqoP1pZ5KMNGqcrzKN95CE0BP-X_XVHg9Jko86g2LWd2OMaCorB2xXRPSPVHlS07NKnzgwoujN9MT8OsftGT02kh1jSPc41S0VAarhnX252pEFPGawReX7vTLTHkAQZxAq8ORPj1lhOG_uD0b7w5cacSwc7f15Ovt55rMokSzd2_t-c5Xp_E-zmme_FHKAk89aURJoxeeIOhtncBULlAPzD2VsdQ2Jjr5OuIWuizmYgh-Y2SQKte-l-t7ZTp9GLcKvKwpSkBvIeEX0Lsq8H9_-Va62MVasySiIVh2woVt3WG8fc6BadKzrHIGnKza-JTAvFuLMp_xuwx_x6qiZs3A9ywh-Oc8Ou4iBqnxD4w6NmPD11mzYmyj22ExH1FCAOL3YOw9HFRxW1JLgDkFXYAX0YSx8M8cls8oTQfBFyDoApNszD8JGBCgWtNvKUYrZl6pd_3Dap5UemoHoX66LJKUkMcwwk2rjH21CcPC9434L0fys5Fn-sWwYGTHzGaynmQxwU5E6T4mVp8GzZKPd0Sm1qdxZZNY5ti4sLrOOS1OjRjTzgPaLcV-jgko18LlzmiHcbgiUhvWTJ_awMNDyXlHXquGoteIqT_1vKG7GhZXm7BwQcEvLFii8UjSlgRG5_r3ZMhDlWlLPmo98adk4ie_bxchoQqhBrf-IAC5wzvjZpL1NcWvfGu_YuLlZPoKNrT6oxLfy_atfjOrj5978NxIW8Xbsc-HfGECxVBBmI2L_ErI8Z1YUcQwUr9JYRuQQaviYBxtBIcV5EekStA4BIYQo-l5UifyKO4NB8vykrM47PJFfVRSN5RZkfAYkGAmNPd7U5O_Ui8LjW9LY2GwGYP5HZxeucXAFHVESemSs7wJWBrlM4yQ0scltkVSR-B5fLsfkt0Ku5-gQcX1jpw0GNOQ4-bEYW2MnxBqA9zByPHLLWwVX8M3nZgiHBnBN_Asu6FeI-tWVHnmeZVPBTph5_IiF43IRXI6gfyBn1l22eJdG4Yems5k17sJ1839pxeclaWIQQCqXAE_qJgr-UX0T0rzA1dJDnZlXoLlFb195klqoO-NnuDi7nUzujdSvXOkpby0gXA.kV186RnBSQS7dzdQflSCtQ"
# リフレッシュトークンを使用できるか検証します。
test_cli = jquantsapi.Client(refresh_token=refresh_token)
try:
    id_token = test_cli.get_id_token()
    if len(id_token) > 0:
        print("refresh_tokenは正常です。次の手順に進んでください。")
except HTTPError:
    print("refresh_tokenを使用できません。再度値を確認してください。")

## データのダウンロードおよび保存

銘柄一覧および、上記で設定したデータ取得期間(start_dt から end_dt) の
全銘柄の価格と財務情報データを取得します。加えて、取得したデータを保存しておきます。データ処理方法を変えたりしても再度データ取得しなくて良くなります。

このステップの実行にはデータを取得する期間によって数十分ほど時間がかかります。

In [ ]:
cli = jquantsapi.Client(refresh_token=refresh_token)

cli.MAX_WORKERS = 10

# 銘柄一覧
if os.path.isfile(raw_stock_list_csvfile_path):
    print(f"すでに{raw_stock_list_csvfile_path}が存在するのでデータ取得をスキップします。")
else:
    stock_list_load: pd.DataFrame = cli.get_list()
    stock_list_load.to_csv(raw_stock_list_csvfile_path, compression="gzip", index=False)
stock_list_load: pd.DataFrame = pd.read_csv(raw_stock_list_csvfile_path, dtype=str)


# 株価情報
if os.path.isfile(raw_stock_price_csvfile_path):
    print(f"すでに{raw_stock_price_csvfile_path}が存在するのでデータ取得をスキップします。")
else:
    stock_price_load: pd.DataFrame = cli.get_price_range(start_dt=start_dt, end_dt=end_dt)
    stock_price_load.to_csv(raw_stock_price_csvfile_path, compression="gzip", index=False)
stock_price_load: pd.DataFrame = pd.read_csv(raw_stock_price_csvfile_path, dtype=str)

# 財務情報

if os.path.isfile(raw_stock_fins_csvfile_path):
    print(f"すでに{raw_stock_fins_csvfile_path}が存在するのでデータ取得をスキップします。")
else:
    stock_fin_load: pd.DataFrame = cli.get_statements_range(start_dt=start_dt, end_dt=end_dt)
    stock_fin_load.to_csv(raw_stock_fins_csvfile_path, compression="gzip", index=False)
stock_fin_load: pd.DataFrame = pd.read_csv(raw_stock_fins_csvfile_path, dtype=str)

In [ ]:
# 上記で保存したファイルを削除するためには以下のコマンドを実行します
# 以降のセルでエラーが出た際にファイルを削除することをためしてみてください。
# os.remove(raw_stock_list_csvfile_path)
# os.remove(raw_stock_price_csvfile_path)
# os.remove(raw_stock_fins_csvfile_path)

## データ処理

過去にJ-Quants株式分析チュートリアルやデータ分析コンペティションで用いられたデータとの互換性をなるべく取るため、データの処理を行います。

In [ ]:
# stock_price(株価)

# 株価情報のobject型をdatetime64[ns]型に変換
stock_price_load["Date"] = pd.to_datetime(stock_price_load["Date"])

# 株価情報のいくつかがobject型になっているので数値型に変換
stock_price_load["Open"] = stock_price_load["Open"].astype(np.float64)
stock_price_load["High"] = stock_price_load["High"].astype(np.float64)
stock_price_load["Low"] = stock_price_load["Low"].astype(np.float64)
stock_price_load["Close"] = stock_price_load["Close"].astype(np.float64)
stock_price_load["Volume"] = stock_price_load["Volume"].astype(np.float64)
stock_price_load["AdjustmentOpen"] = stock_price_load["AdjustmentOpen"].astype(np.float64)
stock_price_load["AdjustmentHigh"] = stock_price_load["AdjustmentHigh"].astype(np.float64)
stock_price_load["AdjustmentLow"] = stock_price_load["AdjustmentLow"].astype(np.float64)
stock_price_load["AdjustmentClose"] = stock_price_load["AdjustmentClose"].astype(np.float64)
stock_price_load["AdjustmentFactor"] = stock_price_load["AdjustmentFactor"].astype(np.float64)
stock_price_load["AdjustmentVolume"] = stock_price_load["AdjustmentVolume"].astype(np.float64)
stock_price_load["TurnoverValue"] = stock_price_load["TurnoverValue"].astype(np.float64)

# 累積調整係数を作成します
# memo: 調整後終値とは https://support.yahoo-net.jp/PccFinance/s/article/H000006678
def generate_cumulative_adjustment_factor(df):
   # 分割併合等の係数を適用日に変更
   df.loc[:, "AdjustmentFactor"] = df["AdjustmentFactor"].shift(-1).fillna(1.0)
 
   # 調整係数を作成するために逆順にソートする
   df = df.sort_values("Date", ascending=False)
   # 累積株価調整係数を作成
   df.loc[:, "EndOfDayQuote CumulativeAdjustmentFactor"] = 1 / df["AdjustmentFactor"].cumprod()
   # ソート順を昇順にする
   df = df.sort_values("Date")
 
   return df
# 累積調整係数を追加
stock_price_load = stock_price_load.sort_values(["Code", "Date"])
stock_price_load = stock_price_load.groupby("Code", group_keys=True).apply(generate_cumulative_adjustment_factor).reset_index(drop=True)

# 普通株 (5桁で末尾が0) の銘柄コードを4桁にします
stock_price_load.loc[(stock_price_load["Code"].str.len() == 5) & (stock_price_load["Code"].str[-1] == "0"), "Code"] = stock_price_load.loc[(stock_price_load["Code"].str.len() == 5) & (stock_price_load["Code"].str[-1] == "0"), "Code"].str[:-1]
stock_price_load["Code"] = stock_price_load["Code"].astype(int)

In [ ]:
# stock_fin(財務情報)

# 財務情報のいくつかがobject型になっているので数値型に変換
numeric_cols_fin = [
    'NetSales', 'OperatingProfit', 'OrdinaryProfit', 'Profit', 'EarningsPerShare', 'DilutedEarningsPerShare', 'TotalAssets', 'Equity', 'EquityToAssetRatio', 
    'BookValuePerShare', 'CashFlowsFromOperatingActivities', 'CashFlowsFromInvestingActivities', 'CashFlowsFromFinancingActivities', 'CashAndEquivalents',
    'AverageNumberOfShares', 'ResultDividendPerShare1stQuarter','ResultDividendPerShare2ndQuarter','ResultDividendPerShare3rdQuarter',
    'ResultDividendPerShareFiscalYearEnd', 'ResultDividendPerShareAnnual','ResultTotalDividendPaidAnnual', 'ResultPayoutRatioAnnual',
    'ForecastDividendPerShare1stQuarter', 'ForecastDividendPerShare2ndQuarter', 'ForecastDividendPerShare3rdQuarter', 'ForecastDividendPerShareFiscalYearEnd',
    'ForecastDividendPerShareAnnual', 'ForecastEarningsPerShare', 'ForecastTotalDividendPaidAnnual', 'ForecastPayoutRatioAnnual',
    'ForecastNetSales', 'ForecastOperatingProfit', 'ForecastOrdinaryProfit', 'ForecastProfit', 'NextYearForecastNetSales', 'NextYearForecastOperatingProfit',
    'NextYearForecastOrdinaryProfit', 'NextYearForecastProfit', 'NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock'
]
stock_fin_load[numeric_cols_fin] = stock_fin_load[numeric_cols_fin].apply(pd.to_numeric, errors='coerce', axis=1)

# 財務情報のobject型をdatetime64[ns]型に変換
stock_fin_load["DisclosedDate"] = pd.to_datetime(stock_fin_load["DisclosedDate"]) #開示
stock_fin_load["CurrentFiscalYearEndDate"] = pd.to_datetime(stock_fin_load["CurrentFiscalYearEndDate"])  # 当事業年度終了日
stock_fin_load["CurrentFiscalYearStartDate"] = pd.to_datetime(stock_fin_load["CurrentFiscalYearStartDate"])  # 当事業年度開始日
stock_fin_load["CurrentPeriodStartDate"] = pd.to_datetime(stock_fin_load["CurrentPeriodStartDate"]) # 当会計期間終了日
stock_fin_load["CurrentPeriodEndDate"] = pd.to_datetime(stock_fin_load["CurrentPeriodEndDate"]) # 当会計期間終了日
stock_fin_load["NextFiscalYearStartDate"] = pd.to_datetime(stock_fin_load["NextFiscalYearStartDate"]) # 当会計期間終了日
stock_fin_load["NextFiscalYearEndDate"] = pd.to_datetime(stock_fin_load["NextFiscalYearEndDate"]) # 当会計期間終了日

# 財務情報の値を調整します
stock_fin_load["Result_FinancialStatement FiscalYear"] = stock_fin_load["CurrentFiscalYearEndDate"].dt.strftime("%Y")

# 財務情報の同一日に複数レコードが存在することに対応します。
# ある銘柄について同一日に複数の開示が行われた場合レコードが重複します。
# ここでは簡易的に処理するために特定のTypeOfDocumentを削除した後に、開示時間順に並べて一番最後に発表された開示情報を採用しています。
stock_fin_load = stock_fin_load.loc[~stock_fin_load["TypeOfDocument"].isin(["DividendForecastRevision", "EarnForecastRevision", "REITDividendForecastRevision", "REITEarnForecastRevision", "FYFinancialStatements_Consolidated_REIT"])]
stock_fin_load = stock_fin_load.sort_values(["DisclosedDate", "DisclosedTime", "Profit"]).drop_duplicates(subset=["LocalCode", "DisclosedDate"], keep="first")

# 普通株 (5桁で末尾が0) の銘柄コードを4桁にします
stock_fin_load.loc[(stock_fin_load["LocalCode"].str.len() == 5) & (stock_fin_load["LocalCode"].str[-1] == "0"), "LocalCode"] = stock_fin_load.loc[(stock_fin_load["LocalCode"].str.len() == 5) & (stock_fin_load["LocalCode"].str[-1] == "0"), "LocalCode"].str[:-1]
stock_fin_load["LocalCode"] = stock_fin_load["LocalCode"].astype(int)

In [ ]:
# stock_list(銘柄一覧)

# 普通株 (5桁で末尾が0) の銘柄コードを4桁にします
stock_list_load.loc[(stock_list_load["Code"].str.len() == 5) & (stock_list_load["Code"].str[-1] == "0"), "Code"] = stock_list_load.loc[(stock_list_load["Code"].str.len() == 5) & (stock_list_load["Code"].str[-1] == "0"), "Code"].str[:-1]
stock_list_load["Code"] = stock_list_load["Code"].astype(int)

# 銘柄一覧と株価データから予測対象列を作成します
# prediction_target は、プライム(111)、スタンダード(112)、グロース(113)の 2022-01-01 以降の売買代金上位2000銘柄とします
target_list = stock_list_load.loc[stock_list_load["MarketCode"].isin(["0111", "0112", "0113"]), "Code"]
pred_targets = (stock_price_load.loc[(stock_price_load["Date"] >= "2022-01-01") & (stock_price_load["Code"].isin(target_list)), ["Code", "TurnoverValue"]].groupby("Code").sum()).sort_values("TurnoverValue", ascending=False).head(2000).index

stock_list_load["prediction_target"] = stock_list_load["Code"].isin(pred_targets)
# universe_comp2 は、プライム、スタンダード、グロースの 2022-08-01 時点の時価総額上位2000銘柄とします
df_atp = stock_price_load.loc[stock_price_load["Date"] == "2022-08-01", ["Code", "AdjustmentClose"]]
df_atp = df_atp.loc[df_atp["Code"].isin(target_list)]


def _nos(df):
    df = df.sort_values(["DisclosedDate", "DisclosedTime"])
    df = df.ffill()
    df = df.tail(1)
    return df

df_nos = stock_fin_load.groupby("LocalCode")[["DisclosedDate", "DisclosedTime", "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"]].apply(_nos)
df_nos = df_nos.reset_index()
df_mcap = pd.merge(df_atp, df_nos, left_on=["Code"], right_on=["LocalCode"], how="inner")
df_mcap["mcap"] = df_mcap["AdjustmentClose"] * df_mcap["NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"]
universe_comp2 = sorted(df_mcap.sort_values("mcap").tail(2000)["Code"])
stock_list_load["universe_comp2"] = stock_list_load["Code"].isin(universe_comp2)


In [ ]:
# stock_list: データの互換性のための各種列名変換など
stock_list: pd.DataFrame = pd.DataFrame()
stock_list["Local Code"] = stock_list_load["Code"]
stock_list["Name (English)"] = stock_list_load["CompanyName"]  # 英語名の代わりに日本語名を設定
stock_list["Section/Products"] = stock_list_load["MarketCodeName"]
stock_list["33 Sector(Code)"] = stock_list_load["Sector33Code"]
stock_list["33 Sector(Name)"] = stock_list_load["Sector33CodeName"]
stock_list["17 Sector(Code)"] = stock_list_load["Sector17Code"]
stock_list["17 Sector(Name)"] = stock_list_load["Sector17CodeName"]
stock_list["Size (New Index Series)"] = stock_list_load["ScaleCategory"]
stock_list["prediction_target"] = stock_list_load["prediction_target"]
stock_list["universe_comp2"] = stock_list_load["universe_comp2"]

In [ ]:
# stock_price: データの互換性のための各種列名変換など
stock_price: pd.DataFrame = pd.DataFrame()
stock_price["Local Code"] = stock_price_load["Code"]
#stock_price["Date"] = stock_price_load["Date"]
stock_price["base_date"] = stock_price_load["Date"]
stock_price['EndOfDayQuote Date'] = stock_price_load["Date"]
stock_price["EndOfDayQuote Open"] = stock_price_load["AdjustmentOpen"]
stock_price["EndOfDayQuote High"] = stock_price_load["AdjustmentHigh"]
stock_price["EndOfDayQuote Low"] = stock_price_load["AdjustmentLow"]
stock_price["EndOfDayQuote Close"] = stock_price_load["AdjustmentClose"]
stock_price["EndOfDayQuote ExchangeOfficialClose"] = stock_price_load["AdjustmentClose"]
stock_price["EndOfDayQuote Volume"] = stock_price_load["AdjustmentVolume"]
stock_price["EndOfDayQuote CumulativeAdjustmentFactor"] = stock_price_load["EndOfDayQuote CumulativeAdjustmentFactor"]

stock_price.sort_values(["Local Code", "base_date"], inplace=True)
# ExchangeOfficialClose は欠損値がある場合は前日終値とします
stock_price["EndOfDayQuote ExchangeOfficialClose"] = stock_price.groupby(["Local Code"])["EndOfDayQuote ExchangeOfficialClose"].ffill()
# 前日終値の列を終値列から作成
stock_price["EndOfDayQuote PreviousClose"] = stock_price.groupby(["Local Code"])["EndOfDayQuote Close"].shift(1)

In [ ]:
# stock_financial: データの互換性のための各種列名変換など
stock_fin: pd.DataFrame = pd.DataFrame()
stock_fin["Local Code"] = stock_fin_load["LocalCode"]
stock_fin["Result_FinancialStatement FiscalPeriodEnd"] = stock_fin_load["CurrentPeriodEndDate"].dt.strftime("%Y/%m")
stock_fin["Result_FinancialStatement TotalAssets"] = stock_fin_load["TotalAssets"] # 総資産
stock_fin["Result_FinancialStatement NetAssets"] = stock_fin_load["Equity"] # 純資産
stock_fin["Result_FinancialStatement NetSales"] = stock_fin_load["NetSales"] # 純売上高
stock_fin["Result_FinancialStatement OperatingIncome"] = stock_fin_load["OperatingProfit"] # 営業利益
stock_fin["Result_FinancialStatement OrdinaryIncome"] = stock_fin_load["OrdinaryProfit"]  # 経常利益
stock_fin["Result_FinancialStatement NetIncome"] = stock_fin_load["Profit"]  # 当期純利益
stock_fin["Result_FinancialStatement ReportType"] = stock_fin_load["TypeOfCurrentPeriod"] # {"1Q", "2Q", "3Q", "FY"}
stock_fin["Result_FinancialStatement FiscalYear"] = stock_fin_load["Result_FinancialStatement FiscalYear"] 
stock_fin["Result_FinancialStatement EarningsPerShare"] = stock_fin_load["EarningsPerShare"]
stock_fin["Result_FinancialStatement CashFlowsFromOperatingActivities"] = stock_fin_load["CashFlowsFromOperatingActivities"]
stock_fin["Result_FinancialStatement CashFlowsFromInvestingActivities"] = stock_fin_load["CashFlowsFromInvestingActivities"]
stock_fin["Result_FinancialStatement CashFlowsFromFinancingActivities"] = stock_fin_load["CashFlowsFromFinancingActivities"]
stock_fin["base_date"] = stock_fin_load["DisclosedDate"] # 開示日
stock_fin["TypeOfDocument"] = stock_fin_load["TypeOfDocument"] # 書類種別
stock_fin["RetrospectiveRestatement"] = stock_fin_load["RetrospectiveRestatement"] #修正再表示フラグ
stock_fin["Forecast_FinancialStatement FiscalPeriodEnd"] = stock_fin_load["CurrentFiscalYearEndDate"].dt.strftime("%Y/%m")
stock_fin.loc[stock_fin["Result_FinancialStatement ReportType"] == "FY", "Forecast_FinancialStatement FiscalPeriodEnd"] = (
    stock_fin_load["CurrentFiscalYearEndDate"] + pd.Timedelta(365, unit="D")
).dt.strftime("%Y/%m")  # 本決算の場合は次の年度予想なので1年後の日付にします
stock_fin["Forecast_FinancialStatement ReportType"] = "FY"  #  予想は通期固定
stock_fin["Forecast_FinancialStatement NetSales"] = stock_fin_load["ForecastNetSales"]
stock_fin.loc[stock_fin["Result_FinancialStatement ReportType"] == "FY", "Forecast_FinancialStatement NetSales"] = stock_fin_load["NextYearForecastNetSales"]

stock_fin["Forecast_FinancialStatement OperatingIncome"] = stock_fin_load["ForecastOperatingProfit"]
stock_fin.loc[stock_fin["Result_FinancialStatement ReportType"] == "FY", "Forecast_FinancialStatement OperatingIncome"] = stock_fin_load["NextYearForecastOperatingProfit"]

stock_fin["Forecast_FinancialStatement OrdinaryIncome"] = stock_fin_load["ForecastOrdinaryProfit"]
stock_fin.loc[stock_fin["Result_FinancialStatement ReportType"] == "FY", "Forecast_FinancialStatement OrdinaryIncome"] = stock_fin_load["NextYearForecastOrdinaryProfit"]

stock_fin["Forecast_FinancialStatement NetIncome"] = stock_fin_load["ForecastProfit"]
stock_fin.loc[stock_fin["Result_FinancialStatement ReportType"] == "FY", "Forecast_FinancialStatement NetIncome"] = stock_fin_load["NextYearForecastProfit"]

目的変数を作成します。lookaheadsの値など次第で40分程度かかることがあります。

In [ ]:
def create_label_high_low(stock_code:int, target_date:datetime, lookaheads:List[int], stock_price:pd.DataFrame):
   #memo: 例えばtarget_dateが2022-08-01, lookaheads=[1,5,10]だった場合、stock_priceは2022-08-01より10営業日先のデータまではいってないといけない

   #df_price = stock_price.loc[(stock_price["Local Code"] == stock_code) & (stock_price["base_date"] <= target_date)].copy()
   df_price = stock_price.loc[stock_price["Local Code"] == stock_code].copy()

   output_columns = ["base_date", "Local Code"]
   for lookahead in lookaheads:
       output_columns.append("label_date_{}".format(lookahead))
       output_columns.append("label_high_{}".format(lookahead))
       output_columns.append("label_low_{}".format(lookahead))
       t_col = "label_date_{}".format(lookahead)
       df_price.loc[:, t_col] = df_price.loc[:, "base_date"].shift(-lookahead)

   if len(df_price) == 0:
       return pd.DataFrame(None, columns=output_columns)

   for lookahead in lookaheads:
       # df_high_high: base_dateからn営業日の間の高値の最大値
       df_high_high = df_price.loc[:, "EndOfDayQuote High"].rolling(lookahead, min_periods=1).max()
       df_high_high = df_high_high.shift(-lookahead)
       df_high_high_diff = df_high_high - df_price.loc[:, "EndOfDayQuote ExchangeOfficialClose"]
       df_price.loc[:, "label_high_{}".format(lookahead)] = df_high_high_diff / df_price.loc[:, "EndOfDayQuote ExchangeOfficialClose"]

       # df_low_low: base_dateからn営業日の間の安値の最小値
       df_low_low = df_price.loc[:, "EndOfDayQuote Low"].rolling(lookahead, min_periods=1).min()
       df_low_low = df_low_low.shift(-lookahead)
       df_low_low_diff = df_low_low - df_price.loc[:, "EndOfDayQuote ExchangeOfficialClose"]
       df_price.loc[:, "label_low_{}".format(lookahead)] = df_low_low_diff / df_price.loc[:, "EndOfDayQuote ExchangeOfficialClose"]

   df_price.replace(np.inf, np.nan, inplace=True)
   df_price = df_price[df_price["base_date"] <= target_date]
   return df_price.loc[:, output_columns]

def create_delivery_label_high_low(stock_codes:List[int], target_date:pd.Timestamp, lookaheads:List[int], stock_price:pd.DataFrame):
   buff = []
   for stock_code in stock_codes:
       df = create_label_high_low(stock_code, target_date, lookaheads, stock_price)
       buff.append(df)
   df_labels = pd.concat(buff)
   return df_labels

def output_stock_labels(stock_labels_csvfile_path:str, df_labels:pd.DataFrame, output_start_dt, end_dt:datetime, lookaheads:List[int]):
   df_labels = df_labels.set_index("base_date")
   df_labels = df_labels.loc[df_labels.index <= end_dt].copy()
   df_labels.index.name = "base_date"
   df_labels_output = df_labels.loc[(df_labels.index >= output_start_dt) & (df_labels.index <= end_dt)]

   label_output_columns = ["Local Code"]
   for lookahead in lookaheads:
       label_output_columns.append("label_date_{}".format(lookahead))
       label_output_columns.append("label_high_{}".format(lookahead))
       label_output_columns.append("label_low_{}".format(lookahead))

   df_labels_output.to_csv(stock_labels_csvfile_path, compression="gzip", float_format="%.5f", columns=label_output_columns)

stock_codes = sorted(stock_price["Local Code"].unique())

stock_labels = create_delivery_label_high_low(stock_codes, end_dt, lookaheads, stock_price)

## GoogleDriveへのCSV保存

取得したデータをcsv.gz形式で保存します。
stock_priceおよびstock_listが巨大なファイル（非圧縮, 2017-01-01〜2022-07-31の期間だと540MB程度)なので、保存には10分ほどかかることがあります。

In [ ]:
stock_list.to_csv(stock_list_csvfile_path, compression="gzip", index=False)
stock_price.to_csv(stock_price_csvfile_path, compression="gzip", index=False)
stock_fin.to_csv(stock_fins_csvfile_path, compression="gzip", index=False)
output_stock_labels(stock_labels_csvfile_path, stock_labels, start_dt, end_dt, lookaheads)

# J-Quantsチュートリアル

ここからは、取得したデータを用いて J-Quants株式分析チュートリアルの第2章の内容を再現していきます。
https://github.com/JapanExchangeGroup/J-Quants-Tutorial/blob/main/handson/Chapter02/20210121-chapter02-tutorial.ipynb

### コンペティション課題の説明

予測対象は、東証上場企業が、決算短信を発表した後の20営業日の期間における、当該企業の株価の最高値及び最安値です。   
上場企業は決算期末を含め四半期毎に決算内容が定まった際、決算内容の開示が義務付けられています。  
決算の内容として開示される決算短信には財務諸表が添付されており、財務諸表は企業のファンダメンタル情報を含む複数の表で構成されています。

<img src="https://drive.google.com/uc?export=view&id=1JMYkYbbkFUNjiANJPfC49aLLRo7ERVk5">

### データセットの説明

|ファイル名|概要|
|:-|:-|
|stock_list|各銘柄の情報が記録されたデータ|
|stock_price|各銘柄の株価情報（始値・高値・安値・終値等）が記録されたデータ|
|stock_fin|各銘柄のファンダメンタル情報（決算数値データや配当データ等）が記録されたデータ|
|stock_labels|本コンペティションで学習に用いるラベル（目的変数）が記録されたデータ|

### データ読み込み

In [ ]:
# 追加で必要なライブラリのインストール・インポート
!pip install shap

import sys
import seaborn as sns
import shap
import xgboost
from scipy.stats import spearmanr
import matplotlib
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.ensemble import (
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.metrics import accuracy_score, mean_squared_error
import pickle
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

pd.set_option("display.max_columns", None)

In [ ]:
# 保存したファイルからデータの読み込みをする場合はコメントを外してください
stock_list = pd.read_csv(stock_list_csvfile_path)
stock_price = pd.read_csv(stock_price_csvfile_path)
stock_price["base_date"] = pd.to_datetime(stock_price["base_date"])
stock_fin = pd.read_csv(stock_fins_csvfile_path)
stock_fin["base_date"] = pd.to_datetime(stock_fin["base_date"])
stock_labels = pd.read_csv(stock_labels_csvfile_path)
stock_labels["base_date"] = pd.to_datetime(stock_labels["base_date"])

In [ ]:
# 日時を各データフレームのインデックスに設定しておきます
stock_price.set_index("base_date", inplace=True)
stock_fin.set_index("base_date", inplace=True)
stock_labels.set_index("base_date", inplace=True)

### Data Exploring



#### 銘柄情報（stock_list）

stock_listは、銘柄の名前や業種区分などの基本情報が含まれています。  
発行済株式数は、会社が発行することをあらかじめ定款に定めている株式数（授権株式数）のうち、会社が既に発行した株式数のことです。  
発行済株式数と株価とかけ合わせて時価総額を計算することができます。  
時価総額は企業価値を評価する際に用いられる重要な指標です。  
業種区分情報は、マーケットにおける業種別の平均などを計算する時に役立つ情報です。  
33業種は証券コード協議会が定めており、17業種はTOPIX-17シリーズとして「投資利便性を考慮して17業種に再編したもの」(JPX東証33業種別株価指数・TOPIX-17シリーズファクトシートより引用 https://www.jpx.co.jp/markets/indices/line-up/index.html) です。

#### Chapter 2.2.1の表

In [ ]:
print(stock_list.shape)
stock_list.tail(20)

#### 株価情報(stock_price)

stock_priceには各銘柄の各日付の始値や終値などの株価情報が記録されています。  
テクニカル分析などで終値ベースの分析を実施する場合は、ExchangeOfficialCloseを利用します。  
ここでいうテクニカル分析というのは、マーケットデータから計算される指標に基づいた分析のことです。  
また、終値ベースの分析とは、マーケットデータの中でも、終値のみを用いた分析を表しています。

株価情報は、「株式分割」や「株式併合」が発生した際に生じる株価の変動を、株式数の変化率に応じて調整されています。  
特徴量の定義によっては、その日付時点で実際に取引された株価や出来高を取得したい場合がありますが、その場合累積調整係数を使用して  
[調整前株価] = [調整済株価] * [累積調整係数] 及び [調整前出来高] = [調整済出来高] / [累積調整係数]  
という計算で算出可能です。

#### Chapter 2.2.2の表

In [ ]:
print(stock_price.shape)
stock_price.head(10)

#### 財務情報(stock_fin)

株式投資における ファンダメンタル情報 とは、対象銘柄の純資産といった財務状況や当期純利益といった業績状況を表す情報のことです。  
ファンダメンタル情報を用いて、各銘柄の成長性、収益性、安全性、割安度などの投資判断に活用することができます。  
ファンダメンタル情報を利用した解析は、さまざまな手法が考案されています。

ファンダメンタル情報のデータセットであるstock_finにおいて、  
いくつかの変数名は`Forecast`から始まっていますが、これらは各企業が来期の自社の業績・財務状況を予想したデータです。  
例えば、企業が来期の業績が厳しいことが予め分かっている場合には、予想として早めに開示することがあるため、予想のデータも重要な可能性があります。

#### Chapter 2.2.3の表

In [ ]:
print(stock_fin.shape)
stock_fin.head(10)

#### 目的変数(stock_labels)
stock_labelsは予測の目的変数のデータであり、各銘柄で決算発表が行われた日の取引所公式終値から、  
その日の翌営業日以降N（20）営業日間における最高値及び最安値への変化率を記録したデータです。  

各値の計算式は、 ([基準日付の翌日以降N営業日間における高値/安値] / [基準日付の終値]) - 1 です。  
なお、ラベルの対象期間 (20営業日の間) に値が付かなかった場合は、ラベルを NaN としております。

#### Chapter 2.2.5の表

In [ ]:
print(stock_labels.shape)
stock_labels.head(10)

### Visualization

データセットの各項目の特徴を把握することは、モデルを作成する上で重要な要素の1つです。  
一般にデータの特徴を把握するためには、各項目の意味を把握し、値の平均や標準偏差などの基本統計量を確認します。  
可視化もそういった特徴把握の手法の1つで、データをグラフなどで表現することで特性を直感的に理解できるようになります。

#### Chapter 2.5.1 売上高、営業利益、純利益、純資産及びその決算期の間の関係について可視化

In [ ]:
# stock_fin
# 銘柄コード8697にデータを絞る
code = 8697
fin_data = stock_fin[stock_fin["Local Code"] == code]

# 2019年までの値を表示
fin_data = fin_data[:"2022"]

# プロット対象を定義
columns = [
    "Result_FinancialStatement NetSales",  # 売上高
    "Result_FinancialStatement OperatingIncome",  # 営業利益
    "Result_FinancialStatement NetIncome",  # 純利益
    "Result_FinancialStatement NetAssets",  # 純資産
]

# プロット
fig = px.scatter_matrix(
    fin_data,
    dimensions=columns,
    color="Result_FinancialStatement ReportType",
    labels={col: col.split(" ")[-1] for col in columns}
    )
fig.update_traces(diagonal_visible=False)
fig.show()

上記のプロットについて、各色（緑、紫、青、オレンジ）はそれぞれQ1,Q2,Q3,Annualにおける決算の値に対応しており、また、その他のプロットは、各軸2つの変数の散布図を表しています。

例えば、2行1列目のグラフを見ると、横軸が売上高、縦軸が営業利益になっています。  
高い売上高は高い営業利益に繋がっています。

また、決算期がQ1からQ3,本決算に至るまでに基本的に右肩上がりであることが分かります。  
このことから財務データの純売上高や営業利益などの変数は、各決算期ごとの値ではなく、  
各決算期を積み上げ式で記録されていると推測できます。

#### Chapter 2.5.1 複数銘柄のファンダメンタル情報の比較

In [ ]:
# 銘柄コード9984と9983を比較する
codes = [9984, 9983]

multi_df = dict()

# プロット対象を定義
columns = [
    "Result_FinancialStatement NetSales",  # 売上高
    "Result_FinancialStatement OperatingIncome",  # 営業利益
    "Result_FinancialStatement NetIncome",  # 純利益
    "Result_FinancialStatement NetAssets",  # 純資産
    "Result_FinancialStatement ReportType"  # 決算期
]

# 比較対象の銘柄コード毎に処理
for code in codes:
    # 特定の銘柄コードに絞り込み
    fin_data = stock_fin[stock_fin["Local Code"] == code]
    # 2023年までの値を表示
    fin_data = fin_data[:"2023"].copy()
    # 重複を排除
    fin_data.drop_duplicates(
        subset=[
            "Local Code",
            "Result_FinancialStatement FiscalYear",
            "Result_FinancialStatement ReportType"
        ],
        keep="last", inplace=True)
    # プロット対象のカラムを取得
    _fin_data = fin_data[columns]
    # 決算期毎の平均を取得
    multi_df[code] = _fin_data[columns].groupby("Result_FinancialStatement ReportType").mean()

# 銘柄毎に処理していたものを結合
multi_df = pd.concat(multi_df)
# 凡例を調整
multi_df.set_index(multi_df.index.map(lambda t: f"{t[0]}/{t[1]}"), inplace=True)
# プロット
fig = px.bar(multi_df.T, barmode='group')
fig.show()

NetSales（売上高）とNetAssets（純資産）の特性の違いがわかります。  
NetAssetsは一定の数値になっており、決算期の影響をあまり受けていないことがわかります。  
一方、NetSalesはQ1からAnnualにかけて数値が積み上がっており、Q1から決算期が進むごとに大きくなる特性を持つことがわかります。

#### Chapter 2.5.2 株価の終値の動き

ここでは、サンプルとして銘柄コード9984の「ソフトバンクグループ」の終値の動きを可視化します。

In [ ]:
# 特定の銘柄コードに絞り込み
code = 9984
price_data = stock_price[stock_price["Local Code"] == code]
# 2023年までの値を表示
price_data = price_data[:"2023"]

fig = go.Figure(data=[
    go.Scatter(x=price_data.index, y=price_data["EndOfDayQuote ExchangeOfficialClose"], name=f"securities code : {code}.T"),
])
fig.show()

#### Chapter 2.5.3 移動平均

ここでは移動平均をプロットします。  
移動平均にもさまざまな種類がありますが、ここでは単純移動平均線を用います。  
単純移動平均線というのは、例えば、5日線であれば、直近5営業日の価格の平均値です。  
これを1つずつ期間をスライドしながら計算したものになります。

In [ ]:
# 特定の銘柄コードに絞り込み
code = 9984
price_data = stock_price[stock_price["Local Code"] == code]
# 2019年までの値を表示
price_data = price_data[:"2023"].copy()

# 5日、25日、75日の移動平均を算出
periods = [5, 25, 75]
cols = []
for period in periods:
    col = "{} windows simple moving average".format(period)
    price_data[col] = price_data["EndOfDayQuote ExchangeOfficialClose"].rolling(period, min_periods=1).mean()
    cols.append(col)

fig = go.Figure()
for col in cols:
    fig.add_trace(go.Scatter(x=price_data.index, y=price_data[col], name=col))

fig.show()

#### Chapter 2.5.4 価格変化率

価格変化率は、価格がその期間でどれくらい変化したかを(%)で表現したものです。  
相場の勢いや方向性等を判断する際によく使われます。


In [ ]:
# 特定の銘柄コードに絞り込み
code = 9984
price_data = stock_price[stock_price["Local Code"] == code]
# 2019年までの値を表示
price_data = price_data[:"2023"].copy()

# 5日、25日、75日の価格変化率を算出
periods = [5, 25, 75]
cols = []
for period in periods:
    col = "{} windows rate of return".format(period)
    price_data[col] = price_data["EndOfDayQuote ExchangeOfficialClose"].pct_change(period) * 100
    cols.append(col)

# プロット
fig = go.Figure()
for col in cols:
    fig.add_trace(go.Scatter(x=price_data.index, y=price_data[col], name=col))

fig.show()

#### Chapter 2.5.5 ヒストリカル・ボラティリティ

ここではヒストリカル・ボラティリティを計算します。  
ここで計算するヒストリカル・ボラティリティは、5日、25日、75日の対数リターンの標準偏差です。  
ヒストリカル・ボラティリティはリスク指標の一つで、価格がどの程度激しく変動したかを把握するために利用します。  
一般的にヒストリカル・ボラティリティが大きい銘柄は、小さい銘柄よりも資産として保持するリスクが相対的に高いと考えられます。





In [ ]:
# 特定の銘柄コードに絞り込み
code = 9984
price_data = stock_price[stock_price["Local Code"] == code]
# 2019年までの値を表示
price_data = price_data[:"2023"].copy()

# 5日、25日、75日のヒストリカル・ボラティリティを算出
periods = [5, 25, 75]
cols = []
for period in periods:
    col = "{} windows volatility".format(period)
    price_data[col] = np.log(price_data["EndOfDayQuote ExchangeOfficialClose"]).diff().rolling(period).std()
    cols.append(col)

# プロット
fig = go.Figure()
for col in cols:
    fig.add_trace(go.Scatter(x=price_data.index, y=price_data[col], name=col))

fig.show()

#### Chapter 2.5.6 複数のデータを同時にプロット

In [ ]:
from plotly.subplots import make_subplots

# 特定の銘柄コードに絞り込み
code = 9984
price_data = stock_price[stock_price["Local Code"] == code]
# 2019年までの値を表示
price_data = price_data[:"2020"].copy()

# 5日、25日、75日を対象に値を算出
periods = [5, 25, 75]
ma_cols = []
# 移動平均線
for period in periods:
    col = "{} windows simple moving average".format(period)
    price_data[col] = price_data["EndOfDayQuote ExchangeOfficialClose"].rolling(period, min_periods=1).mean()
    ma_cols.append(col)

return_cols = []
# 価格変化率
for period in periods:
    col = "{} windows rate of return".format(period)
    price_data[col] = price_data["EndOfDayQuote ExchangeOfficialClose"].pct_change(period) * 100
    return_cols.append(col)

vol_cols = []
# ヒストリカル・ボラティリティ
for period in periods:
    col = "{} windows volatility".format(period)
    price_data[col] = np.log(price_data["EndOfDayQuote ExchangeOfficialClose"]).diff().rolling(period).std()
    vol_cols.append(col)

# プロット
fig = make_subplots(rows=3, cols=1)

for col in ma_cols:
    fig.append_trace(go.Scatter(
        x=price_data.index,
        y=price_data[col],
        name=col),
        row=1, col=1)

for col in return_cols:
    fig.append_trace(go.Scatter(
        x=price_data.index,
        y=price_data[col],
        name=col),
        row=2, col=1)

for col in vol_cols:
    fig.append_trace(go.Scatter(
        x=price_data.index,
        y=price_data[col],
        name=col),
        row=3, col=1)

fig.show()

ここでは2020年中頃に起きた株価の下落に着目してみます。  
複数の特徴量を並べてプロットすると、株価に大きな変動があった時に他の特徴量にどのような影響を与えているかを観測することができます。

移動平均の特徴量は、期間が短いほど敏感に株価の下落に反応し、期間が長い特徴量ほど反応が遅れることがわかります。  
リターンの特徴量も下落時には同一の傾向が見て取れますが、その後高いリターンが観測されることがわかります。  
一方、ヒストリカル・ボラティリティの挙動をみると、下落の前にじわじわとボラティリティが上昇していることがわかります。  
このように一つの株価の下落を見ても、それぞれの特徴量の挙動が微妙に異なっており、複数個の特徴量をモデルに投入することで、これらの挙動のパターンを学習することが想像できます。

ここまでデータの読み込み及び可視化について説明してきましたが、  
ここからはデータの前処理やモデル構築に関して説明していきます。   
大まかな流れは、次の図のとおりです。


<img src="https://drive.google.com/uc?export=view&id=1NFqTL9C0KH_I1Uacxk2VoEDYTYQkBxUv">


上図のとおり、モデルを構築する際には、データセットをそのまま入力するのではなく、  
欠損値処理や正規化処理などのデータセットの前処理を実施してから入力することが一般的です。  
ここではデータセットの前処理について解説していきます。

#### 欠損値の処理

本コンペティションのデータについて、まずは実際に欠損が発生している箇所やパターンを特定するために、欠損の発生状況をプロットして確認してみます。

#### Chapter 2.6.2


In [ ]:
# memo: stock_finの内容を変更してしまうとセルの実行に途中で失敗してしまったとき厄介なのでなるべく変更しない

# 2019年までの値を表示
fin_data = stock_fin[:"2022"]

# # データ数の確認
print(fin_data.shape)

# # データの欠損値数を確認
print(fin_data.isna().sum())

# 欠損値の数を年別に集計
fin_data = fin_data.isna()
fin_data["year"] = fin_data.index.year

# データの欠損値をプロット
# fig, ax = plt.subplots(figsize=(20, 5))
# sns.heatmap(fin_data.groupby("year").agg("sum"), ax=ax)

fig = px.imshow(fin_data.groupby("year").agg("sum"))
fig.show()

明るい色で示されている箇所は、欠損値が多く発生していることを表しています。  
本チュートリアルでは欠損値について以下のように対応します。
- `Result_FinancialStatement`の `CashFlowsFromOperatingActivities`、 `CashFlowsFromFinancingActivities`、`CashFlowsFromInvestingActivities`に多くの欠損値があります（上図中央）。  
これらのカラムの値は`Result_FinancialStatement ReportType`が`Annual` の場合にのみ値が入っています。  
これらの欠損値については 0 を代入することで対処します。  

ただし、変数によっては、0という数字自体に意味があるケースが有るため、気をつける必要があります。

実際の欠損値処理は、次のように変数の型 (今回は np.float64 という型を選択) 別に後段処理できる値 (今回は 0) で欠損値を埋めます。

In [ ]:
# 銘柄コード9984にデータを絞る
code = 8697
fin_data = stock_fin[stock_fin["Local Code"] == code]

# float64型の列に絞り込み
fin_data = fin_data.select_dtypes(include=["float64"])

# 欠損値を0でフィル
fin_data = fin_data.fillna(0)

In [ ]:
fin_data.head(5).T

### Define features

**（なぜ特徴量の設計が重要なのか）**  
機械学習の手法にはモデルにできるだけ生に近いデータを与えてその関係性を見つける手法とドメイン知識や専門性を活かして、特徴量を設計する手法があります。

前者の手法はEnd-To-End Learningと呼ばれ、生に近いデータをモデルに与え、そのモデル自体に特徴量を発見させる手法です。音声認識などの分野で活用されています。  

本チュートリアルでは、金融データに慣れ親しんでいただくためにも、特徴量の影響を細かく考察しながら汎化性能に貢献する特徴量を設計していくアプローチで、モデルを構築します。

特徴量生成は、仮説を考え、その仮説をモデルが学ぶにはどのような特徴量が必要か、ということを想像することが重要です。

本チュートリアルでは、「直近株価が上がったら、高値もより大きく変動しやすい」という仮説を立て、この仮説を基に特徴量を生成してみます。  
この仮説をモデルが学ぶためには直近株価が上がったことを示す特徴量が必要です。  
直近を仮に1ヶ月と仮定すると、20日リターンや20日移動平均乖離率などが候補になります。

また、この仮説が市場においても必ずしも正しいという必要はなく、仮説を思いついたら、その仮説を学ぶことができる特徴量を想像し、実際に実験してみることが重要です。

1/2/3monthのリターンに加えて標準偏差、移動平均乖離率などの特徴量生成

※時刻は分析対象データを決算後翌営業日から一週間で価格データが存在していた銘柄のみにフィルタリング

**（定常性）**  
時系列データを扱う際には、定常性を意識して特徴量を設計することが重要です。

株価をそのまま学習させたケースと定常性がある特徴量を利用するケースについて考えてみます。

株価をそのまま学習させたケース: 例えば、モデルの訓練期間における株価が、100円〜110円の範囲で動いたとします。もし、この数値をそのままモデルに投入すると、モデルは株価が100円〜110円近辺で動くことを暗黙に学習します。しかし、この暗黙の仮定は実際のマーケットでは成立しておらず、テスト期間で株価が高騰すると、モデルがうまく動かないことがあります。他にも株価に特有の例としては株式分割や株式併合により株価のレンジが大きく変動する場合があります。

定常性がある特徴量を利用するケース: 例えば、20日の価格変化率を考えると、これは正規分布ではありませんが、一部のマーケットの混乱期を除けばほぼ0を中心とした正規分布に近い分布になります。特徴量は、2%の上昇や4%の下落といった0を中心とした時系列となっており、将来に渡っても似たような分布になることが期待でき、株価範囲に対する暗黙の仮定を学ぶ恐れがなくなります。このように将来に渡っても似たような分布を期待できる特徴量は定常性があるといえます。

#### Chapter 2.7.3

ここでは、特徴量の生成を stock_price の株価情報を利用して行います。  
株価情報には、価格や出来高など市場で公開されている株価の四本値(始値、高値、安値、終値)の時系列データが格納されています。  
本チュートリアルでは、特徴量の例として1ヶ月、2ヶ月、3ヶ月間の「終値の変化率（リターン）」、「ヒストリカル・ボラティリティ」、「移動平均線からの乖離率」を紹介します。


In [ ]:
# 銘柄コード9984にデータを絞る
code = 9984
price_data = stock_price[stock_price["Local Code"] == code]

# 終値のみに絞る
feats = price_data[["EndOfDayQuote ExchangeOfficialClose"]].copy()

# 終値の20営業日リターン
feats["return_1month"] = feats["EndOfDayQuote ExchangeOfficialClose"].pct_change(20)
# 終値の40営業日リターン
feats["return_2month"] = feats["EndOfDayQuote ExchangeOfficialClose"].pct_change(40)
# 終値の60営業日リターン
feats["return_3month"] = feats["EndOfDayQuote ExchangeOfficialClose"].pct_change(60)

# 終値の20営業日ボラティリティ
feats["volatility_1month"] = (
    np.log(feats["EndOfDayQuote ExchangeOfficialClose"]).diff().rolling(20).std()
)
# 終値の40営業日ボラティリティ
feats["volatility_2month"] = (
    np.log(feats["EndOfDayQuote ExchangeOfficialClose"]).diff().rolling(40).std()
)
# 終値の60営業日ボラティリティ
feats["volatility_3month"] = (
    np.log(feats["EndOfDayQuote ExchangeOfficialClose"]).diff().rolling(60).std()
)

# 終値と20営業日の単純移動平均線の乖離
feats["MA_gap_1month"] = feats["EndOfDayQuote ExchangeOfficialClose"] / (
    feats["EndOfDayQuote ExchangeOfficialClose"].rolling(20).mean()
)
# 終値と40営業日の単純移動平均線の乖離
feats["MA_gap_2month"] = feats["EndOfDayQuote ExchangeOfficialClose"] / (
    feats["EndOfDayQuote ExchangeOfficialClose"].rolling(40).mean()
)
# 終値と60営業日の単純移動平均線の乖離
feats["MA_gap_3month"] = feats["EndOfDayQuote ExchangeOfficialClose"] / (
    feats["EndOfDayQuote ExchangeOfficialClose"].rolling(60).mean()
)

# 欠損値処理
feats = feats.fillna(0)
# 元データのカラムを削除
feats = feats.drop(["EndOfDayQuote ExchangeOfficialClose"], axis=1)

In [ ]:
feats.tail(5).T

In [ ]:
fig = make_subplots(rows=3, cols=1)

for col in feats.columns[:3]:
    fig.add_trace(go.Scatter(
        x=feats.index,
        y=feats[col],
        name=col
    ),
    row=1, col=1
)
for col in feats.columns[3:6]:
    fig.add_trace(go.Scatter(
        x=feats.index,
        y=feats[col],
        name=col
    ),
    row=2, col=1
)
for col in feats.columns[6:]:
    fig.add_trace(go.Scatter(
        x=feats.index,
        y=feats[col],
        name=col
    ),
    row=3, col=1
)

fig.show()

### バックテスト用のテストデータ作成

ここでは、バックテストを行うためのデータの分割について説明します。

（バックテストとは）
バックテストとは、モデルの有効性を検証する際に、過去のデータを用いて、一定期間にどの程度のパフォーマンスが得られたかをシミュレーションすることです。  
モデルの有効性を検証する上で、どのようにバックテストを実施するかは重要なポイントになります。

ここでは、データセットを訓練データとテストデータに切り分けます。まずデータセットを分ける理由を説明し、次に、データセットの分け方及びコツについて解説します。

データセットを分ける理由は、モデルの汎化性能を確認し使用するモデルを決定するためです。

汎化性能とは、モデルが訓練データ以外の未知のデータに対しても機能するという能力です。  
汎化性能が低いモデルは、訓練データでは高い精度が得られますが、訓練データにない未知のデータについては、低い精度しか得られません。  
この現象を過学習と呼びます。  
データセットを分割せずに、そのまま全体に対して学習し、同じデータセットに対してモデルによる予測をすると、基本的には高い精度の結果を得ることができます。  
しかし、未知のデータに対して予測すると、予測がまったく当たらないということが起こり得ます。  
そのため、データセットを分割して、学習に使用していないデータをモデルの検証用として用意しておくことで、   
作成したモデルが過学習していないことを確認することが出来ます。

基本的な時系列データの分割手法(ホールドアウト検証)に関して解説します。

1. 全体のデータセットを、訓練期間(TRAIN)/検証期間(VAL)/テスト期間(TEST)で分けます。
2. TRAINデータでモデルを学習させ、VALデータでモデルを評価します。これをモデルのさまざまなパラメーターで何度か行い、一番結果が良かったパラメーターを選びます。
3. そして最後にTESTデータでモデルの予測結果を最終評価します。

本チュートリアルでは、次の期間でデータを分割します。

訓練期間： 2018-01-01 - 2019-12-31  
評価期間： 2020-02-01 - 2020-12-01  
テスト期間：2021-01-01 - 2022-12-31  

※データの分割に際し、各期間に間隔（1か月）を空けている理由は、未来の情報を含ませないようにするためです。例えば、2017年12月31日の目的変数には5営業日、10営業日、20営業日後の株価リターンの情報が入っているため、2017年12月31日のデータを使って学習したモデルは未来の情報（2018年1月のリターン）を知っていることになってしまいます。したがって、2018年1月のデータを検証データに含めてしまうとリークが発生し、適切なモデルの評価ができなくなってしまいます。

#### Chapter 2.8.2


In [ ]:
# データ分割期間を定義
TRAIN_END = "2019-12-31"
VAL_START = "2020-02-01"
VAL_END = "2020-12-01"
TEST_START = "2021-01-01"

### 特徴量の生成
ここでは、モデルの学習に用いるためのデータを準備します。

モデル作成のステップは、以下のとおりに行います。

1. 銘柄を一つ選ぶ
2. その銘柄に対して、財務データ及びマーケットデータから特徴量を作る
3. 全銘柄に対して同じことを繰り返す
4. 作成したデータを結合する
5. 全データを訓練データ、評価データ、テストデータに分ける
6. 訓練データで予測モデルを学習させる

#### Chapter 2.9.1

ここでは、特徴量生成のコードを示しています。コードのおおまかな流れとしては、以下の3ステップに分けられます。

1. 財務データの取得及び前処理（Chapter 2.6.2と同様）
2. マーケットデータの取得及び特徴量定義（Chapter 2.7.2と同様）
3. 財務データと生成した特徴量を結合

In [ ]:
def get_features_for_predict(stock_fin, stock_price, code:int, start_dt="2018-01-01"):
    """
    Args:
        stock_fin (pd.DataFrame)  : pd.DataFrame that includes financial statements data
        stock_price (pd.DataFrame) : pd.DataFrame that includes stock price data
        code (int)  : A local code for a listed company
        start_dt (str): specify date range
    Returns:
        feature DataFrame (pd.DataFrame)
    """
    # おおまかな手順の1つ目

    # 特定の銘柄コードのデータに絞る
    fin_data = stock_fin[stock_fin["Local Code"] == code]
    # 特徴量の作成には過去60営業日のデータを使用しているため、
    # 予測対象日からバッファ含めて土日を除く過去90日遡った時点から特徴量を生成します
    n = 90
    # 特徴量の生成対象期間を指定
    fin_data = fin_data.loc[pd.Timestamp(start_dt) - pd.offsets.BDay(n) :]
    # fin_dataのnp.float64のデータのみを取得
    fin_data = fin_data.select_dtypes(include=["float64"])
    # 欠損値処理
    fin_feats = fin_data.fillna(0)

    # おおまかな手順の2つ目

    # 特定の銘柄コードのデータに絞る
    price_data = stock_price[stock_price["Local Code"] == code]
    # 終値のみに絞る
    feats = price_data[["EndOfDayQuote ExchangeOfficialClose"]]
    # 特徴量の生成対象期間を指定
    feats = feats.loc[pd.Timestamp(start_dt) - pd.offsets.BDay(n) :].copy()

    # 終値の20営業日リターン
    feats["return_1month"] = feats["EndOfDayQuote ExchangeOfficialClose"].pct_change(20)
    # 終値の40営業日リターン
    feats["return_2month"] = feats["EndOfDayQuote ExchangeOfficialClose"].pct_change(40)
    # 終値の60営業日リターン
    feats["return_3month"] = feats["EndOfDayQuote ExchangeOfficialClose"].pct_change(60)
    # 終値の20営業日ボラティリティ
    feats["volatility_1month"] = (
        np.log(feats["EndOfDayQuote ExchangeOfficialClose"]).diff().rolling(20).std()
    )
    # 終値の40営業日ボラティリティ
    feats["volatility_2month"] = (
        np.log(feats["EndOfDayQuote ExchangeOfficialClose"]).diff().rolling(40).std()
    )
    # 終値の60営業日ボラティリティ
    feats["volatility_3month"] = (
        np.log(feats["EndOfDayQuote ExchangeOfficialClose"]).diff().rolling(60).std()
    )
    # 終値と20営業日の単純移動平均線の乖離
    feats["MA_gap_1month"] = feats["EndOfDayQuote ExchangeOfficialClose"] / (
        feats["EndOfDayQuote ExchangeOfficialClose"].rolling(20).mean()
    )
    # 終値と40営業日の単純移動平均線の乖離
    feats["MA_gap_2month"] = feats["EndOfDayQuote ExchangeOfficialClose"] / (
        feats["EndOfDayQuote ExchangeOfficialClose"].rolling(40).mean()
    )
    # 終値と60営業日の単純移動平均線の乖離
    feats["MA_gap_3month"] = feats["EndOfDayQuote ExchangeOfficialClose"] / (
        feats["EndOfDayQuote ExchangeOfficialClose"].rolling(60).mean()
    )

    # おおまかな手順の3つ目
    # 欠損値処理
    feats = feats.fillna(0)
    # 元データのカラムを削除
    feats = feats.drop(["EndOfDayQuote ExchangeOfficialClose"], axis=1)

    # 財務データの特徴量とマーケットデータの特徴量のインデックスを合わせる
    feats = feats.loc[feats.index.isin(fin_feats.index)]
    fin_feats = fin_feats.loc[fin_feats.index.isin(feats.index)]

    # データを結合
    feats = pd.concat([feats, fin_feats], axis=1).dropna()

    # 欠損値処理を行います。
    feats = feats.replace([np.inf, -np.inf], 0)

    # 銘柄コードを設定
    feats["code"] = code

    # 生成対象日以降の特徴量に絞る
    feats = feats.loc[pd.Timestamp(start_dt) :]

    return feats

In [ ]:
df = get_features_for_predict(stock_fin, stock_price, 9984)
df.T

### 目的変数の生成
次に目的変数を定義します。目的変数は、データセットの stock_labels 内にあり、利用する際は先ほど定義した特徴量のデータセットに対して、行（日付）を一致させる必要があります。

データセットの訓練期間、評価期間、テスト期間への分割処理も合わせて実施します。

#### Chapter 2.9.2


In [ ]:
def get_features_and_label(stock_labels, codes, feature, label):
    """
    Args:
        stock_labels (pd.DataFame): label data for training
        codes  (array) : target codes
        feature (pd.DataFrame): features
        label (str) : label column name
    Returns:
        train_X (pd.DataFrame): training data
        train_y (pd.DataFrame): label for train_X
        val_X (pd.DataFrame): validation data
        val_y (pd.DataFrame): label for val_X
        test_X (pd.DataFrame): test data
        test_y (pd.DataFrame): label for test_X
    """
    # 分割データ用の変数を定義
    trains_X, vals_X, tests_X = [], [], []
    trains_y, vals_y, tests_y = [], [], []

    # 銘柄コード毎に特徴量を作成
    for code in tqdm(codes):
        # 特徴量取得
        feats = feature[feature["code"] == code]

        # 特定の銘柄コードのデータに絞る
        labels = stock_labels[stock_labels["Local Code"] == code].copy()

        # 特定の目的変数に絞る
        labels = labels[label].copy()
        # nanを削除
        labels.dropna(inplace=True)

        if feats.shape[0] > 0 and labels.shape[0] > 0:
            # 特徴量と目的変数のインデックスを合わせる
            labels = labels.loc[labels.index.isin(feats.index)]
            feats = feats.loc[feats.index.isin(labels.index)]
            labels.index = feats.index

            # データを分割
            _train_X = feats[: TRAIN_END]
            _val_X = feats[VAL_START : VAL_END]
            _test_X = feats[TEST_START :]

            _train_y = labels[: TRAIN_END]
            _val_y = labels[VAL_START : VAL_END]
            _test_y = labels[TEST_START :]

            # データを配列に格納 (後ほど結合するため)
            trains_X.append(_train_X)
            vals_X.append(_val_X)
            tests_X.append(_test_X)

            trains_y.append(_train_y)
            vals_y.append(_val_y)
            tests_y.append(_test_y)

    # 銘柄毎に作成した説明変数データを結合します。
    train_X = pd.concat(trains_X)
    val_X = pd.concat(vals_X)
    test_X = pd.concat(tests_X)
    # 銘柄毎に作成した目的変数データを結合します。
    train_y = pd.concat(trains_y)
    val_y = pd.concat(vals_y)
    test_y = pd.concat(tests_y)

    return train_X, train_y, val_X, val_y, test_X, test_y

In [ ]:
# 対象銘柄コードを定義
codes = [9984]
# 対象の目的変数を定義
label = "label_high_20"
# 特徴量を取得
feat = get_features_for_predict(stock_fin, stock_price, codes[0])
# 特徴量と目的変数を一致させて、データを分割
ret = get_features_and_label(stock_labels, codes, feat, label)
for v in ret:
    print(v.T)

ここまでは一つの銘柄に対して処理をしてきましたが、ここからは全ての予測対象銘柄に対して上記の処理を実施するために、予測対象の銘柄コードを以下のように取得します。

In [ ]:
def get_codes(stock_list):
    """
    Args:
        stock_list (pd.DataFrame): stock list
    Returns:
        array: list of stock codes
    """
    # 予測対象の銘柄コードを取得
    codes = stock_list[stock_list["prediction_target"] == True][
        "Local Code"
    ].values
    return codes

次に、目的変数毎にデータセットを作成します。今回は全ての目的変数に同一の特徴量を使用していますが、目的変数に応じて特徴量をチューニングすることでより精度の高いモデルを作成することができます。



In [ ]:
# 対象の目的変数を定義
labels = {
        "label_high_20",
        "label_low_20",
}

# 目的変数毎にデータを保存するための変数
train_X, val_X, test_X = {}, {}, {}
train_y, val_y, test_y = {}, {}, {}

# 予測対象銘柄を取得
codes = get_codes(stock_list)

# 特徴量を作成
buff = []
for code in tqdm(codes):
    feat = get_features_for_predict(stock_fin, stock_price, code)
    buff.append(feat)
feature = pd.concat(buff)

# 目的変数毎に処理
for label in tqdm(labels):
    # 特徴量と目的変数を取得
    _train_X, _train_y, _val_X, _val_y, _test_X, _test_y = get_features_and_label(stock_labels, codes, feature, label)
    # 目的変数をキーとして値を保存
    train_X[label] = _train_X
    val_X[label] = _val_X
    test_X[label] = _test_X
    train_y[label] = _train_y
    val_y[label] = _val_y
    test_y[label] = _test_y

### Create Model

データの準備が完了したので、いよいよモデルの学習を実行します。  
ここでは、sklearnライブラリのRandomForestRegressorモデルを使用します。  
モデルに設定する各種パラメータは、ここではとくに指定せずにライブラリのデフォルトパラメータを使用します。

RandomForestの回帰モデルであるRandomForestRegressorモデルを利用する理由は、予測する目的変数が連続値であるからです。  
RandomForestモデルは決定木をベースとするモデルであるため、以下の理由から最初に選択するモデルとして扱いやすいです。

RandomForest内部で利用する決定木は、特徴量の大小関係のみに着目しており、値自体には意味がないので正規化処理の必要がありません

特徴量の重要度を取得することができ、次に実施することの道筋を立てやすい

#### Chapter 2.9.3


In [ ]:
# 目的変数を指定
label = "label_high_20"
# モデルの初期化
pred_model = RandomForestRegressor(random_state=0)
# モデルの学習
pred_model.fit(train_X[label], train_y[label])

### Predict

ここでは構築したモデルから予測結果を出力し、可視化などによる分析を実施します。

#### Chapter 2.10.1

In [ ]:
# モデルを定義
models = {
    "rf": RandomForestRegressor,
}

# モデルを選択
model = "rf"

# 目的変数を指定
label = "label_high_20"

# 特徴量グループを定義　
# ファンダメンタル
fundamental_cols = stock_fin.select_dtypes("float64").columns
fundamental_cols = fundamental_cols[fundamental_cols != "Local Code"]
# 価格変化率
returns_cols = [x for x in train_X[label].columns if "return" in x]
# テクニカル
technical_cols = [x for x in train_X[label].columns if (x not in fundamental_cols) and (x != "code")]
columns = {
    "fundamental_only": fundamental_cols,
    "return_only": returns_cols,
    "technical_only": technical_cols,
    "fundamental+technical": list(fundamental_cols) + list(technical_cols),
}

# 特徴量グループを指定
col = "fundamental_only"

# 学習
pred_model = models[model](random_state=0)
print(type(train_X[label][columns[col]].values))
print(type(train_y[label]))
pred_model.fit(train_X[label][columns[col]], train_y[label])

# 予測
result = {}
result[label] = pd.DataFrame(
    pred_model.predict(val_X[label][columns[col]]), columns=["predict"]
)

# 予測結果に日付と銘柄コードを追加
result[label]["datetime"] = val_X[label][columns[col]].index
result[label]["code"] = val_X[label]["code"].values

# 予測の符号を取得
result[label]["predict_dir"] = np.sign(result[label]["predict"])

# 実際の値を追加
result[label]["actual"] = val_y[label].values

予測結果の確認として、実際に決算開示等のあった銘柄について基準日付の終値から最高値への変化率(actual)と予測スコア(predict)の散布図を見ます。  
ここで散布図を選択する理由は、予測対象に対して予測スコアがどのような分布をとっているかを見ることが、  
モデルの挙動を理解するわかりやすい可視化であることが挙げられます。

In [ ]:
from scipy import stats

g = sns.jointplot(data=result[label], x="predict", y="actual", kind="reg")
r, p = stats.pearsonr(result[label]["predict"], result[label]["actual"])
phantom, = g.ax_joint.plot([], [], linestyle="", alpha=0)
g.ax_joint.legend([phantom],['r={:f}, p={:f}'.format(r,p)])

この図では、横軸が予測値で、縦軸が真の値です。予測と真の値には、正の相関(0.182723)が見受けられるので、ある程度の相関関係が発生しています。  
一般的なデータで0.183という数字が出てもほぼ無相関に見えますが、金融データでは0.183というスコアは高い部類に入ります。  
このように視覚化すると、予測値と真の値の関係性を可視化できます。

### 重要度

予測精度を向上させるためには、特徴量とモデルの分析を集中的に行う必要があります。  
特徴量の分析では、さまざまな手法がありますが、ここでは特徴量の重要度の分析とSHAPという手法を紹介します。



#### Chapter 2.11.1

特徴量の重要度は、Random ForestやGradient Boostingなどのいくつかの機械学習モデルで取得でき、  
モデル内でどの程度それぞれの説明変数が、目的変数に対して重要であるかを判断するために参考になる指標です。

重要度が極端に低いものは、そもそも説明変数から除外したり、重要度が高いものは更に分析することで、  
性能の向上が期待できないか、など分析の道筋をつける上でも役に立ちます。

ここではファンダメンタル情報を用いて、  
モデルの訓練データ(2016年初から2017年末まで)における特徴量の重要度を調査します。

次の方法に従って、特徴量の重要度をプロットします。



In [ ]:
rf = pred_model

# プロット
fig, ax = plt.subplots(figsize=(8, 8))
sorted_idx = rf.feature_importances_.argsort()
ax.barh(fundamental_cols[sorted_idx], rf.feature_importances_[sorted_idx])
ax.set_xlabel("Random Forest Feature Importance")

上記の可視化により、 一番上にある NetAssets(純資産) にモデルが注目していることがわかります。  
純資産は資本金や利益剰余金などを合算した指標で、次に登場する TotalAssets(総資産)から負債を引いたものとなります。  
TotalAssets(総資産) は、流動資産や固定資産、繰延資産など、会社の全ての資産を合算したものを示す指標です。
この2つは会社の規模を示す代表的な指標となっています。

Random Forestモデルの内部で、この2つを利用した分岐が多数存在していることを示しており、  
会社規模が重要な指標である可能性を示唆しています。

### SHAP分析

SHAPは、学習済みモデルにおいて、各特徴量がモデルの出力する予測値に与えた影響度を算出してくれるものです。

ここでは、サンプルモデルとしてXGBoostモデルを利用し、 label_high_20 という目的変数に対して、  
どの特徴量が学習に効果的な特徴量なのかを見てみます。

#### Chapter 2.11.2


In [ ]:
# xgboostモデル学習　
train_X_code_int = train_X["label_high_20"].copy()
train_X_code_int["code"] = train_X_code_int["code"].astype(int)
sample_model = xgboost.train({"learning_rate": 0.01}, xgboost.DMatrix(train_X_code_int, label=train_y["label_high_20"]), 100)

In [ ]:
shap.initjs()

explainer = shap.TreeExplainer(model=sample_model, feature_perturbation='tree_path_dependent', model_output='raw')

shap_values = explainer.shap_values(X=train_X_code_int)

shap.summary_plot(shap_values, train_X_code_int, plot_type="bar")

次にshapのsummary_plotを確認します。これは特徴量を少し変化させた時の学習のインパクトを表しています。

In [ ]:
shap.summary_plot(shap_values, train_X_code_int)

この図の見方ですが、上にある特徴量ほどモデルにとって重要であることを意味します。  
色が赤いのがその特徴量が高い時、青いのがその特徴量が低い時のSHAP値になります。  
図からは例えば以下のようなことが読み取れます。

NetAssetsとTotalAssetsがモデルに影響を与える特徴量であることがわかります。  
特にプラス方向に青い色が多いので、これらの値が小さい場合にこの特徴量をプラスに活用していることがわかります。 

さらにvolatility_1monthやvolatility_3monthがモデルに大きな影響を与える特徴量であることがわかります。  
この特徴量は赤い時にプラス方向(高値が大きくなる)の影響が大きいことがわかります。
これはボラティリティが上昇すると高値が高くなるということを意味するので直感に合致します。

MA_gap_1_month が小さい時にプラス方向(高値が大きくなる)に影響を与えています。移動平均乖離率が小さいときは移動平均線がその時点の株価よりも下にいる期間なので、その時に高値が伸びるのは株価が反転している可能性が高いのかもしれません。

上記のような考察を行いながら、さまざまな特徴量を考え、モデルを改善していくことが重要です。

### Check all the patterns

ここでは、複数モデルを用いて、予測及び結果の比較を行いたいと思います。
今回はシンプルなモデルを複数用います。

|モデル名|パラメータ|
|-|-|
|RandomForestRegressor|random_state = 0|
|ExtraTreesRegressor|random_state = 0|
|GradientBoostingRegressor|random_state = 0|

次は学習用のデータセットも複数用意します。

|モデル名|パラメータ|
|-|-|
|fundamental_only|財務諸表データのみ|
|return_only|価格変化率のデータのみ|
|technical_only|テクニカル指標のみ|
|fundamental+technical|財務諸表とテクニカル指標の両方|

#### Chapter 2.12.1


In [ ]:
# モデルを定義
models = {
    "rf": RandomForestRegressor,
    "extraTree": ExtraTreesRegressor,
    "gbr": GradientBoostingRegressor,
}

# 学習用データセット定義
columns = {
    "fundamental_only": fundamental_cols,
    "return_only": returns_cols,
    "technical_only": technical_cols,
    "fundamental+technical": list(fundamental_cols) + list(technical_cols),
}

# 学習済みモデル保存用
trained_models = dict()
# 結果保存用
all_results = dict()
# モデル毎に処理
for model in tqdm(models.keys()):
    all_results[model] = dict()
    trained_models[model] = dict()
    # データセット毎に処理
    for col in tqdm(columns.keys()):
        result = dict()
        trained_models[model][col] = dict()
        # 目的変数毎に処理
        for label in tqdm(labels):
            if len(test_X[label][columns[col]]) > 0:
                # モデル取得
                pred_model = models[model](random_state=0)
                # 学習
                pred_model.fit(train_X[label][columns[col]], train_y[label])
                # 学習済みモデル保存
                trained_models[model][col][label] = pred_model
                # 結果データ作成
                result[label] = test_X[label][["code"]].copy()
                result[label]["datetime"] = test_X[label][columns[col]].index
                # 予測
                result[label]["predict"] = pred_model.predict(test_X[label][columns[col]])
                result[label]["predict_dir"] = np.sign(result[label]["predict"])
                # 実際の結果
                result[label]["actual"] = test_y[label].values
                result[label]["actual_dir"] = np.sign(result[label]["actual"])
                result[label].dropna(inplace=True)

        all_results[model][col] = result

In [ ]:
results = []
for model in all_results.keys():
    for col in all_results[model]:
        tmp = pd.concat(all_results[model][col])
        tmp["model"] = model
        tmp["feature"] = col
        results.append(tmp)
results = pd.concat(results)
results["label"] = [x[0] for x in results.index]
results["id"] = results["code"].astype(str)+results["datetime"].dt.strftime('%Y%m%d')
results.head(5)

#### Chapter2.12.2

まずは、今回用いる評価関数のリストを紹介します。

|||
|-|-|
|RMSE|二乗平均平方根|
|accuracy|目的変数の符号と予測した目的変数の符号の精度|
|spearman_corr|スピアマンの順位相関|
|corr|ピアソンの相関係数|
|R^2 score|単回帰した時の直線と観測値のバラつき|


In [ ]:
all_metrics = []

for feature in columns:
    matrix = dict()
    for model in models:
        for label in labels:
            tmp_df = results[(results["model"] == model) & (results["label"] == label) & (results["feature"] == feature)]
            rmse = np.sqrt(mean_squared_error(tmp_df["predict"], tmp_df["actual"]))
            accuracy = accuracy_score(tmp_df["predict_dir"], tmp_df["actual_dir"])
            corr = np.corrcoef(tmp_df["actual"], tmp_df["predict"])[0, 1]
            spearman_corr = spearmanr(tmp_df["actual"], tmp_df["predict"])[0]
            matrix[label] = [rmse, accuracy, spearman_corr,corr, corr**2, feature, model, tmp_df.shape[0]]
        res = pd.DataFrame.from_dict(matrix).T
        res.columns = ["RMSE","accuracy","spearman_corr","corr","R^2 score","feature", "model", "# of samples"]
        all_metrics.append(res)
all_metrics = pd.concat(all_metrics)
all_metrics.reset_index()

In [ ]:
numeric_cols = ["RMSE","accuracy","spearman_corr","corr","R^2 score"]
for col in numeric_cols:
    all_metrics[col] = all_metrics[col].astype(float)
agg = all_metrics.reset_index().groupby(["index","feature"]).agg("mean")
agg

この表のテクニカル分析（technical_only）とファンダメンタルデータ（fundamental_only）にそれぞれ着目すると、  
高値の予測の場合には、テクニカル分析のみを用いた特徴量の方が、ファンダメンタルデータよりも精度（accuracy）という観点で若干優れていることが分かります。  
逆に安値の予測の場合には、ファンダメンタルデータのみを用いた特徴量の方が若干優れています。  

また、テクニカル分析とファンダメンタルデータの両方を特徴量を用いた場合の結果に関しては、テクニカル分析よりも若干全体正解率が高くなっていますが、誤差の範囲内です。相関係数（corr）に関し、テクニカル分析とファンダメンタルデータを両方用いた場合は、用いていない場合に比べて、優れていることが分かります。

このように特徴量を選択しながら複数のモデルをつくり、複数の評価関数で評価を行うことで、テクニカル分析とファンダメンタルデータを組み合わせたアプローチのポテンシャルが高いことがわかります。

### Submit Model

本ノートブックでは、コンペ開催中に必要であったモデルの保存とサブミット用パッケージの作成は実施いたしません。

In [ ]:
# # モデル保存用にメソッドを定義します
# def save_model(model, label, model_path="../model"):
#     # モデル保存先ディレクトリを作成
#     os.makedirs(model_path, exist_ok=True)
#     with open(os.path.join(model_path, f"my_model_{label}.pkl"), "wb") as f:
#         # モデルをpickle形式で保存
#         pickle.dump(model, f)

In [ ]:
# # 保存した学習済みモデルから、提出するモデルを選択してpickle形式で保存します。
# # 使用するモデルや特徴量を変更する際は、学習時と推論時で同一の特徴量をモデルに
# # 入力するために提出用のpredictor.pyについても変更する必要があることにご注意ください。

# # モデルの保存先を指定します。
# model_path = f"{STORAGE_DIR_PATH}/model"
# # モデルの種類
# models = ["rf"]
# # 使用する特徴量カラム
# columns = ["fundamental+technical"]
# # 目的変数
# labels = [
#     "label_high_20",
#     "label_low_20",
# ]

# # モデル毎に処理
# for model in models:
#     # 特徴量毎に処理
#     for col in columns:
#         # 目的変数毎に処理
#         for label in labels:
#             # 学習済みモデルを取得
#             pred_model = trained_models[model][col][label]
#             # モデルを保存
#             save_model(pred_model, label, model_path=model_path)

これでこちらのチュートリアル、ならびにこのノートブックは終了になります。
お疲れ様でした！